
# Scikit-learn Processes: Data Prep → Modeling → Evaluation (1–2 hours)

**Goal:** Hands-on tour of the *end-to-end* scikit-learn workflow using a small, **census-like** synthetic dataset with both numeric and categorical features.

**What you'll learn**
- Create a toy dataset that resembles census microdata
- Train/test split & baselines
- Build preprocessing pipelines: imputation, scaling, one-hot encoding
- Fit models (logistic regression, SGD with different **loss** functions, random forest)
- Evaluate with multiple metrics (accuracy, F1, ROC AUC, confusion matrix)
- Cross-validation & model selection with **GridSearchCV**
- Save & load trained pipelines
- (Optional) short regression mini-example, including choices of regression losses

> 💡 You can later swap in your **real census** CSV and keep the same pipeline structure.


In [ ]:

from __future__ import annotations

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler, MinMaxScaler
from sklearn.impute import SimpleImputer

from sklearn.linear_model import LogisticRegression, SGDClassifier, Ridge, SGDRegressor, LinearRegression
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor

from sklearn.metrics import (
    accuracy_score, f1_score, roc_auc_score, RocCurveDisplay,
    ConfusionMatrixDisplay, classification_report,
    mean_absolute_error, mean_squared_error, r2_score
)

import joblib
import warnings
warnings.filterwarnings("ignore")

RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)
pd.set_option("display.max_columns", 100)



## 1) Build a small census-like dataset

We'll simulate a binary **employment_status** target:
- `1` = employed, `0` = not employed

**Features**
- Numeric: `age`, `household_size`
- Categorical: `sex` (`M`/`F`), `education` (`none`, `primary`, `secondary`, `tertiary`), `region` (`north`, `center`, `south`)
- Binary service access: `electricity` (`yes`/`no`)

We'll also inject a few missing values to practice imputation.


In [ ]:

n = 900

age = np.random.randint(15, 70, size=n)
household_size = np.clip(np.round(np.random.normal(4.5, 1.8, size=n)), 1, 12).astype(int)

sex = np.random.choice(["M", "F"], size=n, p=[0.48, 0.52])
education = np.random.choice(["none", "primary", "secondary", "tertiary"],
                             size=n, p=[0.18, 0.38, 0.34, 0.10])
region = np.random.choice(["north", "center", "south"], size=n, p=[0.2, 0.45, 0.35])
electricity = np.random.choice(["yes", "no"], size=n, p=[0.72, 0.28])

# Latent propensity for employment (log-odds)
logit = (
    -3.0
    + 0.06 * age
    - 0.12 * (household_size - 4)
    + np.select(
        [
            education == "none",
            education == "primary",
            education == "secondary",
            education == "tertiary",
        ],
        [-0.8, -0.2, 0.4, 1.0], default=0.0
    )
    + np.where(sex == "M", 0.2, 0.0)
    + np.where(electricity == "yes", 0.4, -0.1)
    + np.select([region=="north", region=="center", region=="south"], [0.1, 0.0, -0.05])
)

# Convert to probability with logistic
p = 1 / (1 + np.exp(-logit))
employed = (np.random.rand(n) < p).astype(int)

df = pd.DataFrame({
    "age": age,
    "household_size": household_size,
    "sex": sex,
    "education": education,
    "region": region,
    "electricity": electricity,
    "employed": employed
})

# Introduce some missingness
for col in ["age", "household_size", "sex", "education", "electricity"]:
    mask = np.random.rand(n) < 0.05
    df.loc[mask, col] = np.nan

df.head()



## 2) Train/test split & baseline

We keep a **test set** for honest evaluation.
We'll also compute a majority-class baseline.


In [ ]:

from sklearn.dummy import DummyClassifier

X = df.drop(columns=["employed"])
y = df["employed"]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=RANDOM_STATE, stratify=y
)

baseline = DummyClassifier(strategy="most_frequent", random_state=RANDOM_STATE)
baseline.fit(X_train, y_train)
y_base = baseline.predict(X_test)
print("Baseline accuracy:", round(accuracy_score(y_test, y_base), 3))
print("Baseline F1:", round(f1_score(y_test, y_base, zero_division=0), 3))



## 3) Preprocessing pipelines

- **Numeric**: median imputation → scaling (we'll try both `StandardScaler` and `MinMaxScaler` later)
- **Categorical**: most-frequent imputation → one-hot encoding

We use a `ColumnTransformer` to apply the right transforms to the right columns.


In [ ]:

num_cols = ["age", "household_size"]
cat_cols = ["sex", "education", "region", "electricity"]

numeric_standard = Pipeline([
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler", StandardScaler())
])

numeric_minmax = Pipeline([
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler", MinMaxScaler())
])

categorical = Pipeline([
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("onehot", OneHotEncoder(handle_unknown="ignore"))
])

preprocess_standard = ColumnTransformer([
    ("num", numeric_standard, num_cols),
    ("cat", categorical, cat_cols)
])

preprocess_minmax = ColumnTransformer([
    ("num", numeric_minmax, num_cols),
    ("cat", categorical, cat_cols)
])



## 4) Fit first model: Logistic Regression (log-loss)

Logistic Regression implicitly optimizes **logistic loss** (a.k.a. log loss).  
We'll measure Accuracy, F1, and AUC.


In [ ]:

logreg_pipe = Pipeline([
    ("prep", preprocess_standard),
    ("clf", LogisticRegression(max_iter=1000, random_state=RANDOM_STATE))
])

logreg_pipe.fit(X_train, y_train)
y_pred = logreg_pipe.predict(X_test)
y_prob = logreg_pipe.predict_proba(X_test)[:, 1]

print("LogReg accuracy:", round(accuracy_score(y_test, y_pred), 3))
print("LogReg F1:", round(f1_score(y_test, y_pred), 3))
print("LogReg ROC AUC:", round(roc_auc_score(y_test, y_prob), 3))

print("\nClassification report:\n", classification_report(y_test, y_pred, digits=3))

disp = ConfusionMatrixDisplay.from_estimator(logreg_pipe, X_test, y_test)
plt.title("Confusion Matrix: Logistic Regression")
plt.show()

RocCurveDisplay.from_estimator(logreg_pipe, X_test, y_test)
plt.title("ROC Curve: Logistic Regression")
plt.show()



## 5) Exploring **loss functions** with `SGDClassifier`

`SGDClassifier` supports multiple loss functions:
- `"log_loss"` → logistic regression (probabilistic, good for AUC/LogLoss)
- `"hinge"` → linear SVM (max-margin, works well with scaled features)
- `"modified_huber"` → robust hinge-like loss with probabilities

We'll compare via cross-validation.


In [ ]:

sgd_losses = ["log_loss", "hinge", "modified_huber"]
cv_results = {}
for loss in sgd_losses:
    sgd_pipe = Pipeline([
        ("prep", preprocess_standard),
        ("clf", SGDClassifier(loss=loss, random_state=RANDOM_STATE, max_iter=2000, tol=1e-3))
    ])
    # For hinge we evaluate accuracy; for log_loss/modified_huber we can also do ROC AUC
    scores = cross_val_score(sgd_pipe, X, y, cv=5, scoring="accuracy")
    cv_results[loss] = scores

for loss, scores in cv_results.items():
    print(f"Loss={loss:>14} | CV Accuracy: {scores.mean():.3f} ± {scores.std():.3f}")



## 6) Model selection with `GridSearchCV`

We'll search:
- Preprocessing scaler (`Standard` vs `MinMax`)
- Classifier type (Logistic Regression vs SGD)
- SGD **loss** function & regularization `alpha`

> Note: This is a small space so it runs fast.


In [ ]:

pipe = Pipeline([
    ("prep", preprocess_standard),
    ("clf", LogisticRegression(max_iter=1000, random_state=RANDOM_STATE))
])

param_grid = [
    {
        "prep": [preprocess_standard, preprocess_minmax],
        "clf": [LogisticRegression(max_iter=1000, random_state=RANDOM_STATE)],
        "clf__C": [0.5, 1.0, 2.0]
    },
    {
        "prep": [preprocess_standard, preprocess_minmax],
        "clf": [SGDClassifier(random_state=RANDOM_STATE, max_iter=2000, tol=1e-3)],
        "clf__loss": ["log_loss", "hinge", "modified_huber"],
        "clf__alpha": [1e-4, 1e-3, 1e-2]
    }
]

grid = GridSearchCV(
    pipe, param_grid=param_grid, cv=5, scoring="f1", n_jobs=None, refit=True
)
grid.fit(X_train, y_train)

print("Best params:", grid.best_params_)
print("Best CV F1:", round(grid.best_score_, 3))

best_model = grid.best_estimator_
yhat = best_model.predict(X_test)
yhat_prob = None
try:
    yhat_prob = best_model.predict_proba(X_test)[:, 1]
except Exception:
    pass

print("\nTest metrics with best model:")
print("Accuracy:", round(accuracy_score(y_test, yhat), 3))
print("F1:", round(f1_score(y_test, yhat), 3))
if yhat_prob is not None:
    print("ROC AUC:", round(roc_auc_score(y_test, yhat_prob), 3))

ConfusionMatrixDisplay.from_estimator(best_model, X_test, y_test)
plt.title("Confusion Matrix: Best Model")
plt.show()



## 7) Tree-based model (no scaling needed): RandomForest

Good to contrast with linear models. Trees are scale-invariant and often strong baselines.


In [ ]:

rf_pipe = Pipeline([
    ("prep", ColumnTransformer([
        ("num", SimpleImputer(strategy="median"), num_cols),
        ("cat", Pipeline([
            ("imputer", SimpleImputer(strategy="most_frequent")),
            ("onehot", OneHotEncoder(handle_unknown="ignore"))
        ]), cat_cols)
    ])),
    ("rf", RandomForestClassifier(n_estimators=200, random_state=RANDOM_STATE))
])

rf_pipe.fit(X_train, y_train)
pred_rf = rf_pipe.predict(X_test)
print("RandomForest accuracy:", round(accuracy_score(y_test, pred_rf), 3))
print("RandomForest F1:", round(f1_score(y_test, pred_rf), 3))



## 8) Save & load the trained pipeline


In [ ]:

model_path = "/mnt/data/employment_best_pipeline.joblib"
joblib.dump(best_model, model_path)

loaded = joblib.load(model_path)
sample = X_test.iloc[[0]]
print("Sample row:\n", sample)
print("\nLoaded model prediction:", loaded.predict(sample)[0])



---

## (Optional) Mini Regression Example: choosing losses

We'll predict a synthetic **wealth_index** and compare losses:
- `LinearRegression` (optimizes **MSE**)
- `Ridge` (MSE + L2 penalty)
- `SGDRegressor` with `loss="squared_error"` (MSE) vs `loss="huber"` (robust to outliers)


In [ ]:

# Create a numeric 'wealth_index' linked to features
rng = np.random.default_rng(RANDOM_STATE)
wealth = (
    0.03 * df["age"].fillna(df["age"].median()).to_numpy()
    + 0.5 * (df["education"] == "secondary").fillna(False).to_numpy().astype(float)
    + 1.0 * (df["education"] == "tertiary").fillna(False).to_numpy().astype(float)
    + 0.6 * (df["electricity"] == "yes").fillna(False).to_numpy().astype(float)
    - 0.05 * df["household_size"].fillna(df["household_size"].median()).to_numpy()
    + rng.normal(0, 0.5, size=len(df))
)

df_reg = df.copy()
df_reg["wealth_index"] = wealth

Xr = df_reg.drop(columns=["wealth_index", "employed"])
yr = df_reg["wealth_index"]

Xr_train, Xr_test, yr_train, yr_test = train_test_split(
    Xr, yr, test_size=0.2, random_state=RANDOM_STATE
)

reg_prep = ColumnTransformer([
    ("num", SimpleImputer(strategy="median"), num_cols),
    ("cat", Pipeline([
        ("imputer", SimpleImputer(strategy="most_frequent")),
        ("onehot", OneHotEncoder(handle_unknown="ignore"))
    ]), cat_cols)
])

# LinearRegression
reg_lr = Pipeline([("prep", reg_prep), ("reg", LinearRegression())])
reg_lr.fit(Xr_train, yr_train)
pred_lr = reg_lr.predict(Xr_test)

# Ridge
reg_ridge = Pipeline([("prep", reg_prep), ("reg", Ridge(alpha=1.0, random_state=RANDOM_STATE))])
reg_ridge.fit(Xr_train, yr_train)
pred_ridge = reg_ridge.predict(Xr_test)

# SGDRegressor (MSE)
reg_sgd_mse = Pipeline([
    ("prep", reg_prep),
    ("reg", SGDRegressor(loss="squared_error", random_state=RANDOM_STATE, max_iter=2000, tol=1e-3))
])
reg_sgd_mse.fit(Xr_train, yr_train)
pred_sgd_mse = reg_sgd_mse.predict(Xr_test)

# SGDRegressor (Huber)
reg_sgd_huber = Pipeline([
    ("prep", reg_prep),
    ("reg", SGDRegressor(loss="huber", random_state=RANDOM_STATE, max_iter=2000, tol=1e-3))
])
reg_sgd_huber.fit(Xr_train, yr_train)
pred_sgd_huber = reg_sgd_huber.predict(Xr_test)

def eval_reg(y_true, y_hat, name):
    mae = mean_absolute_error(y_true, y_hat)
    rmse = mean_squared_error(y_true, y_hat, squared=False)
    r2 = r2_score(y_true, y_hat)
    print(f"{name:>15} | MAE={mae:.3f} | RMSE={rmse:.3f} | R^2={r2:.3f}")

print("Regression comparison:")
eval_reg(yr_test, pred_lr, "LinearRegression")
eval_reg(yr_test, pred_ridge, "Ridge")
eval_reg(yr_test, pred_sgd_mse, "SGD (MSE)")
eval_reg(yr_test, pred_sgd_huber, "SGD (Huber)")


In [ ]:

# Simple true vs predicted plot for one model
plt.figure()
plt.scatter(yr_test, pred_ridge)
plt.xlabel("True wealth_index")
plt.ylabel("Predicted wealth_index")
plt.title("Regression: True vs Predicted (Ridge)")
plt.show()



## Cheat-sheet: choosing losses & scalers

- **Classification**
  - **`log_loss`** (Logistic Regression/SGD): probabilistic outputs; supports AUC/LogLoss metrics; great default.
  - **`hinge`** (Linear SVM via SGD): max-margin; works well with **scaled** features; no probabilities by default.
  - **`modified_huber`**: robust hinge-like; gives usable probabilities.
- **Regression**
  - **MSE / squared_error**: default; sensitive to outliers; pairs well with Ridge/Lasso.
  - **Huber**: robust to outliers; good default when you expect noise.
- **Scaling**
  - **StandardScaler**: centers to 0 mean, unit variance; good for linear models/SGD/SVM.
  - **MinMaxScaler**: squashes to [0, 1]; useful for bounded features or when algorithms assume a range.

**Practical tips**
- Always **impute** missing values inside the pipeline.
- Use **`ColumnTransformer`** to keep numeric vs categorical paths clean.
- Start with a **simple baseline** and **cross-validation**.
- Compare a **linear** model with a **tree** model.
- Save the **whole pipeline** (preprocessing + model).
